In [9]:

import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By

from selenium.webdriver.common.keys import Keys
import time
import re


In [10]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

In [11]:
url = "https://www.lacasaencendida.es/actividades?h=true"
driver.get(url)

In [12]:

driver.find_element(By.CSS_SELECTOR,'#CybotCookiebotDialogBodyLevelButtonLevelOptinAllowallSelection').click()

In [13]:
def get_artist(title):
    splitpatt = r'\,\sde\s|\,\scon\s?'
    try:
        art = re.split(splitpatt,title)[1]
        thistitle = re.split(splitpatt,title)[0]
    except:
        art = "No definido"
        thistitle = title
    return thistitle, art


In [14]:

def collect_page(driver,url):
    data_dict = dict() 
    description = re.split(r':\s|\n',driver.find_element(By.CLASS_NAME,'item-detail__details').text)
    data_dict["title"] = get_artist(driver.find_element(By.CLASS_NAME,'item-detail__hero__info__title').text)[0]
    data_dict["artist"] = get_artist(driver.find_element(By.CLASS_NAME,'item-detail__hero__info__title').text)[1]
    data_dict["date"] = driver.find_element(By.CLASS_NAME,'item-detail__hero__info__when').text
    try:
        data_dict["description"] = driver.find_element(By.CLASS_NAME,'item-detail__title').find_element(By.TAG_NAME,'p').text
    except:
        data_dict["description"] = driver.find_element(By.CLASS_NAME,'item-detail__title').text
    data_dict["text"] = driver.find_element(By.CLASS_NAME,'item-detail__info__content').text
    data_dict["tags"] = driver.find_element(By.CLASS_NAME,'item-detail__info__tags').text.split(', ')
    try:
        data_dict["group"] = driver.find_element(By.CLASS_NAME,'group-link').text
    except:
        data_dict["group"] = "Not in a group"
    for i in range(0,len(description)-1,2):
        data_dict[description[i]] = description[i+1]
    try:
        data_dict["event"] = driver.find_element(By.CLASS_NAME,'item-detail__list').text
    except:
        data_dict["event"] = "No details"
    
    data_dict["category"] = driver.find_element(By.CLASS_NAME,'tags').text.split(', ')
    data_dict["url"] = url
    data_dict["image"] = re.split(r'\s',driver.find_element(By.CLASS_NAME,'slide__image').get_attribute('srcset'))[0]
    return pd.DataFrame([data_dict])
    

# Loop por cada página de la primera lista

In [15]:
def extract_items(npage):
    data_df = pd.DataFrame()
    results = driver.find_elements(By.CLASS_NAME, 'results-list__item')
    for i in range(len(results)):
        results = driver.find_elements(By.CLASS_NAME, 'results-list__item')
        try:
            url_n = results[i].find_element(By.TAG_NAME,'a').get_attribute('href')
            driver.get(url_n)
            new_data = collect_page(driver,url_n)
            time.sleep(0.2) # a 0.5 funciona
            data_df = pd.concat([data_df,new_data],ignore_index=True,axis=0)

            print(i,"de ",len(results),"completado")
            driver.back()
            time.sleep(0.2) 
        except:
            driver.back()
            continue
    return data_df


In [16]:
def main():
    url = 'https://www.lacasaencendida.es/actividades?h=true'
    final_df = pd.DataFrame()
    driver.get(url)
    for i in range(0,239):
        print("PÁGINA ",i,":")
        driver.get(url+'&page='+str(i))
        time.sleep(0.3) 
        new_df = extract_items(i)
        final_df = pd.concat([final_df,new_df],ignore_index=True,axis=0)
        time.sleep(0.2)
        final_df.to_csv(f"parciales/final1.csv")
    final_df.to_csv(f"parciales/final1.csv")
main()

PÁGINA  0 :
0 de  24 completado
1 de  24 completado
2 de  24 completado
3 de  24 completado
4 de  24 completado
5 de  24 completado
6 de  24 completado
7 de  24 completado
9 de  24 completado
10 de  24 completado
11 de  24 completado
12 de  24 completado
13 de  24 completado
14 de  24 completado
15 de  24 completado
16 de  24 completado
17 de  24 completado
18 de  24 completado
19 de  24 completado
20 de  24 completado
21 de  24 completado
22 de  24 completado
23 de  24 completado


In [131]:
import glob
import os
import seaborn as sns
from matplotlib import pyplot as plt
all_files = glob.glob(os.path.join('data/parciales', "*.csv"))
all_files.sort()
casa_df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)
casa_df = casa_df.drop_duplicates('title').drop(['Unnamed: 0'],axis=1).reset_index(drop=True)
clean_casa_df = casa_df.iloc[0:4275,:].copy()
clean_casa_df.to_csv("data/casa-encendida/casa_enc.csv")



,Precio,date,category
47,4,5 may 2023,"['ENCUENTROS', 'ESCÉNICAS']"
46,4,5 may - 6 may 2023,"['CINE', 'ENCUENTROS']"
44,4,6 may - 7 may 2023,"['CINE', 'EN FAMILIA']"
38,4,11 may 2023,"['CINE', 'ENCUENTROS']"
37,4,12 may - 13 may 2023,"['CINE', 'ENCUENTROS']"
35,4,13 may - 14 may 2023,"['CINE', 'EN FAMILIA']"
34,5,14 may 2023,['EN FAMILIA']
32,4,18 may 2023,"['CINE', 'ENCUENTROS']"
31,4,19 may - 20 may 2023,['CINE']
30,5,20 may - 21 may 2023,['ESCÉNICAS']


In [109]:
clean_casa_df

,title,artist,date,description,text,tags,group,Dirigido a,Ubicación,Duración,Precio,event,category,url,image
0,Sonidos de barrio,Kleber Mendonça Filho,15 jul 2023,La película narra las realidades violentas ocu...,"Sonidos de barrio, de Kleber Mendoça Filho. Br...",['#Audiovisuales'],"La Terraza Magnética 2023. ¡Alucina, vecina!",Público general,Terraza,1 hora 59 minutos,4€,No details,['CINE'],https://www.lacasaencendida.es/cine/sonidos-de...,https://www.lacasaencendida.es/storage/14817/r...
1,Cashu,No definido,14 jul 2023,"Carolina Schutzer, alias DJ Cashu, es una de l...",En un intento de ocupar y descentralizar la es...,['#Música'],"La Terraza Magnética 2023. ¡Alucina, vecina!",Público general,Terraza,1 hora,8€,Apertura de puertas a las 20:30 h.,['CONCIERTOS'],https://www.lacasaencendida.es/conciertos/cashu,https://www.lacasaencendida.es/storage/15099/r...
2,El orden divino,Petra Biondina Volpe,13 jul 2023,"En los años setenta, las mujeres no podían vot...","El orden divino, de Petra Biondina Volpe, Suiz...",['#Feminismos'],Gestos necesarios para cambiar el mundo,Público general,Sala Audiovisual,2 horas 30 minutos,4€,"Coloquio con: \nMarifé Santiago Bolaños, poeta...","['CINE', 'ENCUENTROS']",https://www.lacasaencendida.es/cine/el-orden-d...,https://www.lacasaencendida.es/storage/13824/r...
3,Loving,Jeff Nichols,11 jul 2023,En la tercera sesión del ciclo 'Gestos necesar...,"Loving, de Jeff Nichols, EEUU, 2016. 120 min. ...",['#Antirracismo / migración'],Gestos necesarios para cambiar el mundo,Público general,Sala Audiovisual,2 horas 30 minutos,4€,"Coloquio con: \nMoha Gerehou, periodista y exp...","['CINE', 'ENCUENTROS']",https://www.lacasaencendida.es/cine/loving-de-...,https://www.lacasaencendida.es/storage/13815/r...
4,Greener Grass,Jocelyn DeBoer y Dawn Luebbe,8 jul 2023,Considerada una de las piezas más imaginativas...,"Greener Grass, de Jocelyn DeBoer y Dawn Luebbe...",['#Audiovisuales'],"La Terraza Magnética 2023. ¡Alucina, vecina!",Público general,Terraza,1 hora 45 minutos,4€,No details,['CINE'],https://www.lacasaencendida.es/cine/greener-gr...,https://www.lacasaencendida.es/storage/14816/r...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4270,Premio de fotografía humanitaria Luis Valtueña...,No definido,22 ene - 28 ene 2012,Alessandro Grassani recoge la dura vida de los...,NaN,[''],RADIO - Archivo de entrevistas,NaN,NaN,NaN,NaN,No details,['ENCUENTROS'],https://www.lacasaencendida.es/encuentros/prem...,https://www.lacasaencendida.es/storage/11143/r...
4271,"Exposición ""El Paisaje Holandés""",No definido,22 ene - 28 ene 2012,"En los 60 en Holanda, un grupo de artistas cul...",NaN,[''],RADIO - Archivo de entrevistas,NaN,NaN,NaN,NaN,No details,['ENCUENTROS'],https://www.lacasaencendida.es/encuentros/expo...,https://www.lacasaencendida.es/storage/11353/r...
4272,Entrevista a Teresa Aranguren,No definido,15 ene - 21 ene 2012,La periodista analiza la situación del pueblo ...,NaN,[''],RADIO - Archivo de entrevistas,NaN,NaN,NaN,NaN,No details,['ENCUENTROS'],https://www.lacasaencendida.es/encuentros/entr...,https://www.lacasaencendida.es/storage/11154/r...
4273,Entrevista a Jonas Mekas,No definido,15 ene - 21 ene 2012,Un diario íntimo grabado en imágenes. El cinea...,NaN,[''],RADIO - Archivo de entrevistas,NaN,NaN,NaN,NaN,No details,['ENCUENTROS'],https://www.lacasaencendida.es/encuentros/entr...,https://www.lacasaencendida.es/storage/11155/r...
